# 1. Data_preprocessing

In [ ]:
import os
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
sns.set(rc={'figure.dpi': 600}) # set dpi 300
import matplotlib.pyplot as plt
import dataframe_image as dfi

import CAMS_downscale.preprocessing as pre
import test_pnc_model.model_test as model_test
from scikeras.wrappers import KerasRegressor
from keras.optimizers import Adam
from tensorflow import keras

# feature_folder = '../../pncEstimator-main/data/NABEL/feature_data/' # workstation
feature_folder = '../../pncEstimator-main/data/NABEL/feature_eng/'
pncdata2016_2019 = pd.read_csv(feature_folder + 'feature_data_2016_2019.csv')
pncdata2020 = pd.read_csv(feature_folder + 'feature_data_2020.csv')
pncdata2016_2019 = pncdata2016_2019.iloc[:,1:]
pncdata2020 = pncdata2020.iloc[:,1:]
pncdata2016_2019 = pncdata2016_2019[['Date/time', 'station', 'CPC [1/cm3]', 'NOX [ug/m3 eq. NO2]',
                   'PM10 [ug/m3]', 'PM2.5/PM10 ratio', 'O3 [ug/m3]',
                   'Radiation[W/m2]', 'Temperature', 'Precipitation[mm]',
                   'Relative humidity[%]', 'Wind speed[m/s]', 'trafficVol', 'hour', 'month', 'weekday']]
pncdata2020 = pncdata2020[['Date/time', 'station', 'CPC [1/cm3]', 'NOX [ug/m3 eq. NO2]',
                   'PM10 [ug/m3]', 'PM2.5/PM10 ratio', 'O3 [ug/m3]',
                   'Radiation[W/m2]', 'Temperature', 'Precipitation[mm]',
                   'Relative humidity[%]', 'Wind speed[m/s]', 'trafficVol', 'hour', 'month', 'weekday']]

In [7]:
from sklearn.preprocessing import StandardScaler,  MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np

# Split the data into train and test in time (2016-2019 for train, 2020 for test)
x_train = pncdata2016_2019.iloc[:,3:].values
y_train = pncdata2016_2019.iloc[:,2].values
x_test = pncdata2020.iloc[:,3:].values
y_test = pncdata2020.iloc[:,2].values
x_val, x_train, y_val, y_train = train_test_split(x_train, y_train, test_size=0.05, random_state=42)
print('The train dataset ratio = ', y_train.shape[0]/ (y_test.shape[0]+y_train.shape[0]+y_val.shape[0]))
print('The validation dataset ratio = ', y_val.shape[0]/ (y_test.shape[0]+y_train.shape[0]+y_val.shape[0]))
print('The test dataset ratio = ', y_test.shape[0]/ (y_test.shape[0]+y_train.shape[0]+y_val.shape[0]))
scaler_std = StandardScaler()
x_all = np.vstack((x_train, x_val, x_test))
print(x_all.shape, x_train.shape[0], x_val.shape[0], x_test.shape[0])
x_scaler = scaler_std.fit(x_train)
x_train = x_scaler.transform(x_train)
x_val = x_scaler.transform(x_val)
x_test = x_scaler.transform(x_test)

The train dataset ratio =  0.0390648180277497
The validation dataset ratio =  0.742221484644127
The test dataset ratio =  0.21871369732812335
(198849, 13) 7768 147590 43491


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, explained_variance_score
import time
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV, cross_val_score,cross_val_predict,cross_validate, RandomizedSearchCV
from skopt import BayesSearchCV
# from statsmodels.graphics.tsaplots import plot_acf
# import statsmodels.api as sm
import pylab
import warnings
warnings.filterwarnings("ignore")
sns.set(rc={'figure.dpi': 600})

evsall = []
maeall = []
mseall = []
r2all = []



def pred_plot(y_test, y_pred, resid):
    print('----------------------------------')
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    evs = explained_variance_score(y_test, y_pred)
    print('MSE: ', mse)
    print('MAE: ', mae)
    print('r2 score: ', r2)
    print('Explained_variance: ', evs)
    return mse,mae,r2,evs
def out_table(mse,mae,r2,evs):
    evsall.append(evs)
    maeall.append(mae)
    mseall.append(mse)
    r2all.append(r2)
def predpnc(model_fit, x_test, y_test, pncdata2020):
    pred = model_fit.predict(x_test)
    resid = pred - y_test
    mse, mae, r2, evs = pred_plot(y_test, pred, resid)
    out_table(mse, mae, r2, evs)
    pred = pd.DataFrame(pred)
    resid = pd.DataFrame(resid)
    data2020_pred = pd.concat([pncdata2020, pred, resid], axis=1)
    return data2020_pred
def table_result(evsall, maeall, mseall, r2all):
    evsall = pd.DataFrame(evsall)
    maeall = pd.DataFrame(maeall)
    mseall = pd.DataFrame(mseall)
    r2all = pd.DataFrame(r2all)
    table_result = pd.concat([evsall, maeall, mseall, r2all], axis=1)
    table_result.columns = ['evs', 'mae', 'mse', 'r2']
    return table_result

In [4]:
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from joblib import dump

pnc_model_folder = '../out/model/pnc_pred_model/'
pre.create_directories(pnc_model_folder)
# rf_reg = RandomForestRegressor(n_estimators=100, criterion='squared_error', random_state=0, max_depth=100, n_jobs=-1)
# rf_fit2 = rf_reg.fit(x_train, y_train)
# dump(rf_fit2, pnc_model_folder + '/rf_PNC_trainedModel.joblib') 
# lgb_reg = lgb.LGBMRegressor(random_state=42)
# lgb_fit2 = lgb_reg.fit(x_train, y_train)
# dump(lgb_fit2, pnc_model_folder + '/lgb_PNC_trainedModel.joblib') 

# 2. GridsearchCV
- Use GridsearchCV to adjust hyperparameters. (After running this part, need to restart the engine, because we set n_jobs = -1.)

In [ ]:
# 100, 80, 
from sklearn.linear_model import Lasso
param_grid = [{'alpha':[0.1, 1, 10, 25, 50, 80, 100, 500, 1000]}]
lasso_reg = Lasso()
grid_search = GridSearchCV(lasso_reg, param_grid, cv=5)
grid_result = grid_search.fit(x_val, y_val)
print("Best: %f using %s" % (grid_result.best_score_, grid_search.best_params_))
print('-------------------------------------')
means = grid_result.cv_results_['mean_test_score']
params = grid_result.cv_results_['params']
for mean, param in zip(means, params):
    print("%f  with:   %r" % (mean, param))

Best: 0.529425 using {'alpha': 80}
-------------------------------------
0.497830  with:   {'alpha': 0.1}
0.498565  with:   {'alpha': 1}
0.505457  with:   {'alpha': 10}
0.515187  with:   {'alpha': 25}
0.525750  with:   {'alpha': 50}
0.529425  with:   {'alpha': 80}
0.526443  with:   {'alpha': 100}
0.482964  with:   {'alpha': 500}
0.404851  with:   {'alpha': 1000}


In [ ]:
from sklearn import svm

param_grid = {'C':[1, 10, 100], 'gamma':[ 0.5 ,1, 5, 10]}
svm_reg = svm.SVR()
# grid_search = RandomizedSearchCV(svm_reg, param_grid, n_iter=10)
grid_search = GridSearchCV(svm_reg, param_grid, cv=5)
grid_result = grid_search.fit(x_val, y_val)
print("Best: %f using %s" % (grid_result.best_score_, grid_search.best_params_))
print('-------------------------------------')
means = grid_result.cv_results_['mean_test_score']
params = grid_result.cv_results_['params']
for mean, param in zip(means, params):
    print("%f  with:   %r" % (mean, param))

In [ ]:
from sklearn import neighbors

param_grid = {'n_neighbors': [1, 10, 50, 100], 
              'weights': ('uniform', 'distance'),
              'algorithm' : ('auto', 'ball_tree', 'kd_tree', 'brute')
              }
knn = neighbors.KNeighborsRegressor()
grid_search = GridSearchCV(knn, param_grid, cv=5)
grid_result = grid_search.fit(x_val, y_val)
print("Best: %f using %s" % (grid_result.best_score_, grid_search.best_params_))
print('-------------------------------------')
means = grid_result.cv_results_['mean_test_score']
params = grid_result.cv_results_['params']
for mean, param in zip(means, params):
    print("%f  with:   %r" % (mean, param))

Best: 0.398394 using {'algorithm': 'auto', 'n_neighbors': 10, 'weights': 'distance'}
-------------------------------------
0.156545  with:   {'algorithm': 'auto', 'n_neighbors': 1, 'weights': 'uniform'}
0.156545  with:   {'algorithm': 'auto', 'n_neighbors': 1, 'weights': 'distance'}
0.396580  with:   {'algorithm': 'auto', 'n_neighbors': 10, 'weights': 'uniform'}
0.398394  with:   {'algorithm': 'auto', 'n_neighbors': 10, 'weights': 'distance'}
0.394180  with:   {'algorithm': 'auto', 'n_neighbors': 50, 'weights': 'uniform'}
0.397799  with:   {'algorithm': 'auto', 'n_neighbors': 50, 'weights': 'distance'}
0.374476  with:   {'algorithm': 'auto', 'n_neighbors': 100, 'weights': 'uniform'}
0.379194  with:   {'algorithm': 'auto', 'n_neighbors': 100, 'weights': 'distance'}
0.156545  with:   {'algorithm': 'ball_tree', 'n_neighbors': 1, 'weights': 'uniform'}
0.156545  with:   {'algorithm': 'ball_tree', 'n_neighbors': 1, 'weights': 'distance'}
0.396580  with:   {'algorithm': 'ball_tree', 'n_neighb

In [ ]:
from sklearn.tree import DecisionTreeRegressor

param_grid = {
    'criterion': ['mse', 'friedman_mse'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [None, 'sqrt', 'log2']
}

dt = DecisionTreeRegressor(random_state=42)
grid_search = GridSearchCV(estimator=dt, param_grid=param_grid, 
                           scoring='neg_mean_squared_error', 
                           cv=5, n_jobs=20, verbose=2)
grid_result = grid_search.fit(x_val, y_val)
print("Best: %f using %s" % (grid_result.best_score_, grid_search.best_params_))
print('-------------------------------------')
means = grid_result.cv_results_['mean_test_score']
params = grid_result.cv_results_['params']
for mean, param in zip(means, params):
    print("%f  with:   %r" % (mean, param))

Fitting 5 folds for each of 432 candidates, totalling 2160 fits
Best: -44942909.746596 using {'criterion': 'friedman_mse', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'splitter': 'random'}
-------------------------------------
nan  with:   {'criterion': 'mse', 'max_depth': None, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'splitter': 'best'}
nan  with:   {'criterion': 'mse', 'max_depth': None, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'splitter': 'random'}
nan  with:   {'criterion': 'mse', 'max_depth': None, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'splitter': 'best'}
nan  with:   {'criterion': 'mse', 'max_depth': None, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'splitter': 'random'}
nan  with:   {'criterion': 'mse', 'max_depth': None, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 10, 'splitter': 'best'}
nan  with:   {'c

In [ ]:
from sklearn.ensemble import RandomForestRegressor

param_grid = {
    'n_estimators': [10, 50, 100, 200],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
                           scoring='neg_mean_squared_error', 
                           cv=5, n_jobs=20, verbose=2)
grid_result = grid_search.fit(x_val, y_val)
print("Best: %f using %s" % (grid_result.best_score_, grid_search.best_params_))
print('-------------------------------------')
means = grid_result.cv_results_['mean_test_score']
params = grid_result.cv_results_['params']
for mean, param in zip(means, params):
    print("%f  with:   %r" % (mean, param))

Fitting 5 folds for each of 144 candidates, totalling 720 fits
Best: -39629619.057643 using {'max_depth': 20, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 200}
-------------------------------------
-43201662.742232  with:   {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 10}
-40653916.100927  with:   {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
-40466881.400155  with:   {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
-40129332.839862  with:   {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
-42281906.142282  with:   {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 10}
-40434994.946262  with:   {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 50}
-40323555.257995  with:   {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimato

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
param_grid = {
    'base_estimator': [DecisionTreeRegressor(max_depth=d) for d in [1, 2, 3]],
    'n_estimators': [10, 50, 100, 200],
    'learning_rate': [0.01, 0.05, 0.1, 0.5, 1],
    'loss': ['linear', 'square', 'exponential']
}

ada = AdaBoostRegressor(random_state=42)
grid_search = GridSearchCV(estimator=ada, param_grid=param_grid, 
                           scoring='neg_mean_squared_error', 
                           cv=5, n_jobs=20, verbose=2)
grid_result = grid_search.fit(x_val, y_val)
print("Best: %f using %s" % (grid_result.best_score_, grid_search.best_params_))
print('-------------------------------------')
means = grid_result.cv_results_['mean_test_score']
params = grid_result.cv_results_['params']
for mean, param in zip(means, params):
    print("%f  with:   %r" % (mean, param))

Fitting 5 folds for each of 180 candidates, totalling 900 fits
Best: -50560071.933130 using {'base_estimator': DecisionTreeRegressor(max_depth=3), 'learning_rate': 0.05, 'loss': 'exponential', 'n_estimators': 50}
-------------------------------------
-100482570.311520  with:   {'base_estimator': DecisionTreeRegressor(max_depth=1), 'learning_rate': 0.01, 'loss': 'linear', 'n_estimators': 10}
-99018340.116485  with:   {'base_estimator': DecisionTreeRegressor(max_depth=1), 'learning_rate': 0.01, 'loss': 'linear', 'n_estimators': 50}
-99555027.548174  with:   {'base_estimator': DecisionTreeRegressor(max_depth=1), 'learning_rate': 0.01, 'loss': 'linear', 'n_estimators': 100}
-125826425.609986  with:   {'base_estimator': DecisionTreeRegressor(max_depth=1), 'learning_rate': 0.01, 'loss': 'linear', 'n_estimators': 200}
-100201630.806125  with:   {'base_estimator': DecisionTreeRegressor(max_depth=1), 'learning_rate': 0.01, 'loss': 'square', 'n_estimators': 10}
-99897002.504748  with:   {'base_e

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 4, 6],
    'min_samples_leaf': [1, 2, 3],
    'subsample': [0.8, 0.9, 1.0],
    'max_features': [None, 'sqrt', 'log2']
}

gb = GradientBoostingRegressor(random_state=42)
grid_search = GridSearchCV(estimator=gb, param_grid=param_grid, 
                           scoring='neg_mean_squared_error', 
                           cv=5, n_jobs=20, verbose=2)
grid_result = grid_search.fit(x_val, y_val)
print("Best: %f using %s" % (grid_result.best_score_, grid_search.best_params_))
print('-------------------------------------')
means = grid_result.cv_results_['mean_test_score']
params = grid_result.cv_results_['params']
for mean, param in zip(means, params):
    print("%f  with:   %r" % (mean, param))

Fitting 5 folds for each of 2187 candidates, totalling 10935 fits
Best: -37176528.893994 using {'learning_rate': 0.1, 'max_depth': 4, 'max_features': None, 'min_samples_leaf': 2, 'min_samples_split': 6, 'n_estimators': 200, 'subsample': 0.8}
-------------------------------------
-103484636.907827  with:   {'learning_rate': 0.01, 'max_depth': 3, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50, 'subsample': 0.8}
-103603110.421381  with:   {'learning_rate': 0.01, 'max_depth': 3, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50, 'subsample': 0.9}
-103626950.268140  with:   {'learning_rate': 0.01, 'max_depth': 3, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50, 'subsample': 1.0}
-73899178.500156  with:   {'learning_rate': 0.01, 'max_depth': 3, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100, 'subsample': 0.8}
-73954904.257258  with: 

In [ ]:
import lightgbm as lgb
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [20, 40, 60],
    'num_leaves': [31, 61, 91],
    'boosting_type': ['gbdt'],
    'objective': ['regression'],
    'min_data_in_leaf': [20, 30, 40],
    'feature_fraction': [0.8, 0.9, 1.0],
    'bagging_fraction': [0.8, 0.9, 1.0],
    'bagging_freq': [0, 1, 2]
}

lgb_estimator = lgb.LGBMRegressor(boosting_type='gbdt', random_state=42)
grid_search = GridSearchCV(estimator=lgb_estimator, param_grid=param_grid, 
                           scoring='neg_mean_squared_error', 
                           cv=5, n_jobs=20, verbose=2)
grid_result = grid_search.fit(x_val, y_val)
print("Best: %f using %s" % (grid_result.best_score_, grid_search.best_params_))
print('-------------------------------------')
means = grid_result.cv_results_['mean_test_score']
params = grid_result.cv_results_['params']
for mean, param in zip(means, params):
    print("%f  with:   %r" % (mean, param))

Fitting 5 folds for each of 2187 candidates, totalling 10935 fits
[LightGBM] [Warning] min_data_in_leaf is set=40, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=40
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_data_in_leaf is set=40, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=40
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging

# 3. Training
- Use the best params to train the model

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso
from sklearn import neighbors, svm
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, StackingRegressor
import lightgbm as lgb
from sklearn.neural_network import MLPRegressor

linear_reg = LinearRegression().fit(x_train, y_train)
lasso_reg = Lasso(alpha=80).fit(x_train, y_train)
svm_reg = svm.SVR(C=100, gamma=0.5).fit(x_train, y_train)
knn_reg = neighbors.KNeighborsRegressor(n_neighbors=10, weights='distance', n_jobs=20).fit(x_train, y_train)
tree_reg = DecisionTreeRegressor(criterion='friedman_mse', max_depth=10, min_samples_leaf=4, min_samples_split=2,
                                 splitter='random', random_state=42).fit(x_train, y_train)
rf_reg = RandomForestRegressor(n_estimators=200, random_state=42,max_depth=20, max_features=4,
                               bootstrap=False, n_jobs=20).fit(x_train, y_train)
ada_reg = AdaBoostRegressor(random_state=42, n_estimators=50, learning_rate=0.05, loss='exponential').fit(x_train, y_train)
gbr_reg = GradientBoostingRegressor(n_estimators=200, random_state=42, learning_rate=0.1,
                                    max_depth=4, min_samples_leaf=2, min_samples_split=6,
                                    subsample=0.8).fit(x_train, y_train)
lgb_reg = lgb.LGBMRegressor(random_state=42, n_estimators=60, num_leaves=61, objective='regression',
                             n_jobs=20).fit(x_train, y_train)

In [7]:
data2020_pred1 = predpnc(linear_reg, x_test, y_test, pncdata2020)
data2020_pred2 = predpnc(lasso_reg, x_test, y_test, data2020_pred1)
data2020_pred3 = predpnc(svm_reg, x_test, y_test, data2020_pred2)
data2020_pred4 = predpnc(knn_reg, x_test, y_test, data2020_pred3)
data2020_pred5 = predpnc(tree_reg, x_test, y_test, data2020_pred4)
data2020_pred6 = predpnc(rf_reg, x_test, y_test, data2020_pred5)
data2020_pred7 = predpnc(ada_reg, x_test, y_test, data2020_pred6)
data2020_pred8 = predpnc(gbr_reg, x_test, y_test, data2020_pred7)
data2020_pred9 = predpnc(lgb_reg, x_test, y_test, data2020_pred8)
data2020_pred9.columns = ['Date/time', 'station', 'CPC [1/cm3]', 'NOX [ug/m3 eq. NO2]',
                          'PM10 [ug/m3]', 'PM2.5/PM10 ratio', 'O3 [ug/m3]',
                          'Radiation[W/m2]', 'Temperature', 'Precipitation[mm]',
                          'Relative humidity[%]', 'Wind speed[m/s]', 'trafficVol', 'hour',
                          'month', 'weekday', 'linear_pred','linear_resid','lasso_pred','lasso_resid',
                          'svm_pred','svm_resid','knn_pred','knn_resid','tree_pred','tree_resid',
                          'rf_pred','rf_resid','ada_pred','ada_resid','gbr_pred','gbr_resid',
                          'lgb_pred','lgb_resid']
pred_table_path = '../out/pred_table/'
pre.create_directories(pred_table_path)
data2020_pred9.to_csv(pred_table_path + 'pncdata2020_pred_test.csv')

----------------------------------
MSE:  36679398.06906262
MAE:  3641.814619901853
r2 score:  0.7313282043674361
Explained_variance:  0.7356541569309125
----------------------------------
MSE:  37103352.692508586
MAE:  3650.1473512588345
r2 score:  0.728222792175734
Explained_variance:  0.7330063831198541
----------------------------------
MSE:  59442484.61288797
MAE:  4109.795233348063
r2 score:  0.5645915713301723
Explained_variance:  0.5839228394556191
----------------------------------
MSE:  27963955.07967754
MAE:  3307.338767168409
r2 score:  0.7951676848649734
Explained_variance:  0.7955207378207165
----------------------------------
MSE:  28901479.732442893
MAE:  3213.660768091369
r2 score:  0.7883004393492756
Explained_variance:  0.7950430732149709
----------------------------------
MSE:  22524564.17745895
MAE:  2726.6453797955783
r2 score:  0.8350105121135245
Explained_variance:  0.8372388899729392
----------------------------------
MSE:  40788255.078708984
MAE:  3936.45165191

- stack

In [8]:
estimators = [("knn", knn_reg),("rf", rf_reg),("lgb", lgb_reg),("tree",tree_reg)]
final_estimator = MLPRegressor(hidden_layer_sizes=(100,), activation='relu', 
                               solver='adam', random_state=42, max_iter=500)

stack_reg = StackingRegressor(estimators=estimators, final_estimator=final_estimator, cv=5)
stack_pred = stack_reg.fit(x_train, y_train).predict(x_test)
resid = stack_pred - y_test
mse, mae, r2, evs = pred_plot(y_test, stack_pred, resid)

----------------------------------
MSE:  20642921.64767468
MAE:  2833.0705956462643
r2 score:  0.8487932976506225
Explained_variance:  0.8505963474020477


In [10]:
from joblib import dump, load

stack_fit = stack_reg.fit(x_train, y_train)
data2020_pred16 = predpnc(stack_fit, x_test, y_test, data2020_pred9)
data2020_pred16.columns = ['Date/time', 'station', 'CPC [1/cm3]', 'NOX [ug/m3 eq. NO2]',
                           'PM10 [ug/m3]', 'PM2.5/PM10 ratio', 'O3 [ug/m3]',
                           'Radiation[W/m2]', 'Temperature', 'Precipitation[mm]',
                           'Relative humidity[%]', 'Wind speed[m/s]', 'trafficVol', 'hour',
                           'month', 'weekday', 'linear_pred','linear_resid','lasso_pred','lasso_resid',
                           'svm_pred','svm_resid','knn_pred','knn_resid','tree_pred','tree_resid',
                           'rf_pred','rf_resid','ada_pred','ada_resid','gbr_pred','gbr_resid',
                           'lgb_pred','lgb_resid', 'stack_pred','stack_resid']
data2020_pred16.to_csv(pred_table_path + '/pncdata2020_pred.csv')
dump(stack_fit, pnc_model_folder + '/stack_trainedModel.joblib')                  

['../out/model/pnc_pred_model//stack_trainedModel.joblib']

# 4. cross_validation

In [11]:
stations = ['BAS', 'BER', 'HAE', 'LUG', 'RIG']
pncdata2016_2019_0 = pncdata2016_2019[pncdata2016_2019['station'] != stations[0]]
pncdata2016_2019_1 = pncdata2016_2019[pncdata2016_2019['station'] != stations[1]]
pncdata2016_2019_2 = pncdata2016_2019[pncdata2016_2019['station'] != stations[2]]
pncdata2016_2019_3 = pncdata2016_2019[pncdata2016_2019['station'] != stations[3]]
pncdata2016_2019_4 = pncdata2016_2019[pncdata2016_2019['station'] != stations[4]]

In [ ]:
from sklearn.preprocessing import StandardScaler

def crossv(data2016_2019, data2020):
    x_train = data2016_2019.iloc[:,3:].values
    y_train = data2016_2019.iloc[:,2].values
    x_test = data2020.iloc[:,3:].values
    y_test = data2020.iloc[:,2].values
    print(y_train.shape, y_test.shape)
    print('train ratio = ', y_train.shape[0]/ (y_test.shape[0]+y_train.shape[0]))
    print('test ratio = ', y_test.shape[0]/ (y_test.shape[0]+y_train.shape[0]))
    # scaler_minmax = MinMaxScaler()
    scaler_std = StandardScaler()
    x_all = np.vstack((x_train, x_test))
    print(x_all.shape, x_train.shape[0], x_test.shape[0])
    x_scaler = scaler_std.fit(x_train)
    x_train = x_scaler.transform(x_train)
    x_test = x_scaler.transform(x_test)
    
    estimators = [("knn", knn_reg),("rf", rf_reg),("lgb", lgb_reg),("gbr",gbr_reg)]
    final_estimator = MLPRegressor(hidden_layer_sizes=(24, 24), activation='relu', 
                               solver='adam', random_state=42, max_iter=500)
    
    stack_reg = StackingRegressor(estimators=estimators, final_estimator=final_estimator, cv=5)
    stack_fit = stack_reg.fit(x_train, y_train)
    stack_pred = stack_fit.predict(x_test)
    resid = stack_pred - y_test
    mse,mae,r2,evs = pred_plot(y_test, stack_pred, resid)
    stack_pred = pd.DataFrame(stack_pred)
    resid = pd.DataFrame(resid)
    return stack_pred, resid, stack_reg, stack_fit
    

In [20]:
stack_pred0,resid0,stack_reg0, stack_fit0 = crossv(pncdata2016_2019_0, pncdata2020)
data2020_pred17 = pd.concat([data2020_pred16,stack_pred0,resid0],axis=1)
data2020_pred17.columns = ['Date/time', 'station', 'CPC [1/cm3]', 'NOX [ug/m3 eq. NO2]',
                           'PM10 [ug/m3]', 'PM2.5/PM10 ratio', 'O3 [ug/m3]',
                           'Radiation[W/m2]', 'Temperature', 'Precipitation[mm]',
                           'Relative humidity[%]', 'Wind speed[m/s]', 'trafficVol', 'hour',
                           'month', 'weekday', 'linear_pred','linear_resid','lasso_pred','lasso_resid',
                           'svm_pred','svm_resid','knn_pred','knn_resid','tree_pred','tree_resid',
                           'rf_pred','rf_resid','ada_pred','ada_resid','gbr_pred','gbr_resid',
                           'lgb_pred','lgb_resid','stack_pred','stack_resid',
                           'stack0_pred','stack0_resid']
# dump(stack_fit0, model_folder + '/stack_trainedModel0.joblib') 

----------------------------------
MSE:  26874682.237580072
MAE:  3036.192027962939
r2 score:  0.8031464660289684
Explained_variance:  0.8189455855416549


In [15]:
stack_pred1,resid1,stack_reg1, stack_fit1 = crossv(pncdata2016_2019_1, pncdata2020)
data2020_pred18 = pd.concat([data2020_pred17,stack_pred1,resid1],axis=1)
data2020_pred18.columns = ['Date/time', 'station', 'CPC [1/cm3]', 'NOX [ug/m3 eq. NO2]',
                          'PM10 [ug/m3]', 'PM2.5/PM10 ratio', 'O3 [ug/m3]',
                          'Radiation[W/m2]', 'Temperature', 'Precipitation[mm]',
                          'Relative humidity[%]', 'Wind speed[m/s]', 'trafficVol', 'hour',
                          'month', 'weekday', 'linear_pred','linear_resid','lasso_pred','lasso_resid',
                          'svm_pred','svm_resid','knn_pred','knn_resid','tree_pred','tree_resid',
                          'rf_pred','rf_resid','ada_pred','ada_resid','gbr_pred','gbr_resid',
                          'lgb_pred','lgb_resid','stack_pred','stack_resid',
                          'stack0_pred','stack0_resid',
                          'stack1_pred','stack1_resid']
# dump(stack_fit, model_folder + '/stack_trainedModel1.joblib') 

----------------------------------
MSE:  23624417.44674682
MAE:  2788.5322733827397
r2 score:  0.8269542307035752
Explained_variance:  0.8324346521793006


In [16]:
stack_pred2,resid2,stack_reg2, stack_fit2 = crossv(pncdata2016_2019_2, pncdata2020)
data2020_pred19 = pd.concat([data2020_pred18,stack_pred2,resid2],axis=1)
data2020_pred19.columns = ['Date/time', 'station', 'CPC [1/cm3]', 'NOX [ug/m3 eq. NO2]',
                          'PM10 [ug/m3]', 'PM2.5/PM10 ratio', 'O3 [ug/m3]',
                          'Radiation[W/m2]', 'Temperature', 'Precipitation[mm]',
                          'Relative humidity[%]', 'Wind speed[m/s]', 'trafficVol', 'hour',
                          'month', 'weekday', 'linear_pred','linear_resid','lasso_pred','lasso_resid',
                          'svm_pred','svm_resid','knn_pred','knn_resid','tree_pred','tree_resid',
                          'rf_pred','rf_resid','ada_pred','ada_resid','gbr_pred','gbr_resid',
                          'lgb_pred','lgb_resid',
                          'stack_pred','stack_resid','stack0_pred','stack0_resid',
                          'stack1_pred','stack1_resid','stack2_pred','stack2_resid']
# dump(stack_fit2, model_folder + '/stack_trainedModel2.joblib') 

----------------------------------
MSE:  42929322.72724103
MAE:  3326.264252275292
r2 score:  0.6855483233203329
Explained_variance:  0.7028416437249341


In [17]:
stack_pred3,resid3,stack_reg3, stack_fit3 = crossv(pncdata2016_2019_3, pncdata2020)
data2020_pred20 = pd.concat([data2020_pred19,stack_pred3,resid3],axis=1)
data2020_pred20.columns = ['Date/time', 'station', 'CPC [1/cm3]', 'NOX [ug/m3 eq. NO2]',
                           'PM10 [ug/m3]', 'PM2.5/PM10 ratio', 'O3 [ug/m3]',
                           'Radiation[W/m2]', 'Temperature', 'Precipitation[mm]',
                           'Relative humidity[%]', 'Wind speed[m/s]', 'trafficVol', 'hour',
                           'month', 'weekday', 'linear_pred','linear_resid','lasso_pred','lasso_resid',
                           'svm_pred','svm_resid','knn_pred','knn_resid','tree_pred','tree_resid',
                           'rf_pred','rf_resid','ada_pred','ada_resid','gbr_pred','gbr_resid',
                           'lgb_pred','lgb_resid',
                          'stack_pred','stack_resid','stack0_pred','stack0_resid',
                           'stack1_pred','stack1_resid','stack2_pred','stack2_resid',
                           'stack3_pred','stack3_resid']
# dump(stack_fit3, model_folder + '/stack_trainedModel3.joblib') 

----------------------------------
MSE:  24489450.454564307
MAE:  2908.974490176713
r2 score:  0.8206179770100396
Explained_variance:  0.8267259057592479


In [18]:
stack_pred4,resid4,stack_reg4, stack_fit4 = crossv(pncdata2016_2019_4, pncdata2020)
data2020_pred21 = pd.concat([data2020_pred20,stack_pred4,resid4],axis=1)
data2020_pred21.columns = ['Date/time', 'station', 'CPC [1/cm3]', 'NOX [ug/m3 eq. NO2]',
                           'PM10 [ug/m3]', 'PM2.5/PM10 ratio', 'O3 [ug/m3]',
                           'Radiation[W/m2]', 'Temperature', 'Precipitation[mm]',
                           'Relative humidity[%]', 'Wind speed[m/s]', 'trafficVol', 'hour',
                           'month', 'weekday', 'linear_pred','linear_resid','lasso_pred','lasso_resid',
                           'svm_pred','svm_resid','knn_pred','knn_resid','tree_pred','tree_resid',
                           'rf_pred','rf_resid','ada_pred','ada_resid','gbr_pred','gbr_resid',
                           'lgb_pred','lgb_resid',
                          'stack_pred','stack_resid','stack0_pred','stack0_resid',
                           'stack1_pred','stack1_resid','stack2_pred','stack2_resid',
                           'stack3_pred','stack3_resid','stack4_pred','stack4_resid']
# dump(stack_fit4, model_folder + '/stack_trainedModel4.joblib')
# data2020_pred15.to_csv(out_folder + '/crossv_pncdata2020_pred.csv')

----------------------------------
MSE:  22140342.504473034
MAE:  2842.2172550654914
r2 score:  0.8378248856375312
Explained_variance:  0.8389348497698592


In [19]:
data2020_pred21.to_csv(pred_table_path + '/crossv_pncdata2020_pred.csv')